# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [80]:
! pip install -U langchain

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [81]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [82]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

biology_template = """你是一位富有洞察力的生物学家。
你善于用生动的比喻来解释复杂的生物过程。
你对未知领域保持谦逊和好奇的态度。

这是一个问题：
{input}"""

computer_template = """你是一位创新型的计算机科学家。
你擅长将抽象的编程概念与日常生活联系起来。
你鼓励批判性思考和问题解决能力。

这是一个问题：
{input}"""

chinese_literature_template = """你是一位博学的汉语言文学专家。
你善于分析文学作品中的深层含义和文化背景。
你热衷于探讨文学如何反映和影响社会。

这是一个问题：
{input}"""


In [83]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": biology_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": computer_template,
    },
    {
        "name": "汉语言文学",
        "description": "适用于回答汉语言文学问题",
        "prompt_template": chinese_literature_template,
    },
]

In [84]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [85]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

In [86]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [87]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [88]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [89]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '计算机: 适用于回答计算机问题', '汉语言文学: 适用于回答汉语言文学问题']


In [90]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言文学: 适用于回答汉语言文学问题


In [91]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [92]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言文学: 适用于回答汉语言文学问题

<< INPUT >>
{input}

<

In [93]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [94]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？?'}
> Finished chain.
{'input': '黑体辐射是什么？?', 'text': '\n\n黑体辐射是指处于热平衡状态的物体所发出的电磁辐射，也称为热辐射。根据普朗克定律，黑体辐射的强度与温度有关，而且在不同波长处都有不同的强度。经过研究发现，黑体辐射的频谱可以用黑体辐射定律来描述，即普朗克公式。黑体辐射在理论物理和实验物理中都有重要的应用，例如在宇宙背景辐射和太阳辐射中。'}


In [95]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？'}
> Finished chain.
{'input': '大于40的第一个质数是多少，使得这个质数加一能被3整除？', 'text': '\n\n首先，我们需要找出大于40的质数。质数是只能被1和它本身整除的数，所以我们可以从41开始，逐个往上找质数。经过检查，我们发现41是一个质数。\n接下来，我们需要将这个质数加1，得到42。然后我们需要判断42能否被3整除。42除以3等于14，所以42可以被3整除。\n因此，大于40的第一个质数是41，使得这个质数加一为42，能被3整除。'}


In [96]:
router_chain.verbose = True

In [97]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '什么是黑洞？'}
> Finished chain.
{'input': '什么是黑洞？', 'text': '\n\n黑洞是一种极其密集的天体，它的质量非常大，但体积却非常小。它具有极强的引力，甚至连光都无法逃脱。它的存在是由于物质被压缩到极端的程度，使得它的引力场变得非常强大。由于黑洞吞噬周围的物质，所以它的形状看起来就像是一个漆黑的洞。目前，我们还没有直接观测到黑洞，但是它们的存在可以通过它们产生的引力效应来间接证明。 '}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [98]:
# 生物方面的问题
print(chain.invoke("DNA是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
生物: {'input': 'DNA是什么？'}
> Finished chain.
{'input': 'DNA是什么？', 'text': '\nDNA是脱氧核糖核酸（deoxyribonucleic acid）的简称，它是细胞内储存遗传信息的分子。它由四种碱基组成，分别是腺嘌呤（adenine）、鸟嘌呤（guanine）、胸腺嘧啶（thymine）和胞嘧啶（cytosine）。这些碱基按照特定的顺序排列，形成基因，决定了生物的遗传特征和生命活动。DNA也可以通过复制过程来传递遗传信息，保证后代与父母具有相似的遗传特征。'}


In [99]:
# 计算机方面的问题
print(chain.invoke("什么是计算机网络？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
计算机: {'input': '什么是计算机网络？'}
> Finished chain.
{'input': '什么是计算机网络？', 'text': '\n\n计算机网络是一种连接多台计算机的系统，使它们能够相互通信和共享资源。它通过物理连接或无线连接将计算机、服务器、存储设备和其他网络设备连接起来，在全球范围内构成一个巨大的网络。计算机网络可以实现数据传输、远程访问、远程控制、共享资源和通信等功能，为人们提供了便捷的信息交流和资源利用方式。它也是互联网的基础，为我们日常使用的各种应用程序提供了基础设施。'}


In [100]:
# 汉语言文学方面的问题
print(chain.invoke("解释一下《红楼梦》的故事情节。"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
汉语言文学: {'input': '解释一下《红楼梦》的故事情节。'}
> Finished chain.
{'input': '解释一下《红楼梦》的故事情节。', 'text': '\n\n《红楼梦》是一部中国古典长篇章回小说，由清朝作家曹雪芹所著。故事主要围绕着贾宝玉、林黛玉、薛宝钗等人的命运展开。\n\n故事的主要情节是围绕着贾宝玉和林黛玉的爱情展开的。贾宝玉是贾府的公子，林黛玉是林家的女儿，两人从小就认识并相爱。但由于贾宝玉的身份和家族的复杂关系，两人的爱情并不被外人所认可。\n\n随着故事的进展，贾宝玉和林黛玉的爱情遭到了各种挑战和考验。其中包括贾宝玉与'}
